In [1]:
import pickle

import altair as alt

import pandas as pd

import polyclonal

import warnings
warnings.filterwarnings('ignore')

from IPython.utils import io

In [2]:
import os
os.chdir('../../')

In [5]:
serum=3944
test = pd.read_csv(
            f'results/antibody_escape/{serum}_avg.csv'
        ).query(
            "`times_seen` >= 5"
        )

mut_list=['D53G', 'K83E', 'Y94N', 'A128T', 'T131K', 'K135T', 'F137S', 'S138A',
           'H156S', 'Y159N', 'T160I', 'L164Q', 'G186D', 'D190N', 'Y195F',]
filtered_df = test[test['mutation'].isin(mut_list)][['mutation', 'escape_mean']]

filtered_df

,mutation,escape_mean
513,D53G,0.1937
655,K83E,0.0716
731,Y94N,0.0239
930,A128T,-0.0118
950,T131K,-0.0291
989,K135T,0.0090
999,F137S,-0.0643
1001,S138A,-0.2733
1117,H156S,0.0072
1147,Y159N,0.1714


In [6]:
def get_mut_escapes(sera_list, age_group, mut_list):
    escape_df_list = []
    
    for serum in sera_list:
        df = pd.read_csv(
            f'results/antibody_escape/{serum}_avg.csv'
        ).query(
            "`times_seen` >= 5"
        )

        filtered_df = df[df['mutation'].isin(mut_list)][['mutation', 'escape_mean']]
           
        filtered_df['serum'] = serum
        filtered_df['age_group'] = age_group
        
        escape_df_list.append(filtered_df)
        
    mut_escape_df = pd.concat(escape_df_list)
    
    return mut_escape_df

In [31]:
# initialize list of mutations in Darwin/2021
mut_list=['D53G', 'K83E', 'Y94N', 'A128T', 'T131K', 'K135T', 'F137S', 'S138A',
           'H156S', 'Y159N', 'T160I', 'L164Q', 'G186D', 'D190N', 'Y195F', 'K276R']

peds = [2367, 3944, 2462, 2389, 2323, 2388, 3973, 4299, 4584]
teens = [2350, 2365, 2382, 3866, 2380, 3856, 3857, 3862]
adults = ['33C', '34C', '197C', '199C', '215C', '210C', '74C', '68C', '150C', '18C']
# ferrets = ['ferret_1', 'ferret_2', 'ferret_3']

sample_lists = [peds, teens, adults]
cohorts = ['0-5 years', '15-18 years', '40-45 years']
mut_escape_dfs = []

i=0 # for looping through age cohort definitions

# start by getting full escape df filtered to key sites
for list in sample_lists:
    mut_escape_df = get_mut_escapes(list, cohorts[i], mut_list)
    mut_escape_dfs.append(mut_escape_df)

    i+=1

full_escape = pd.concat(mut_escape_dfs)

full_escape['serum'] = full_escape['serum'].astype(str)

full_escape

,mutation,escape_mean,serum,age_group
514,D53G,0.0968,2367,0-5 years
657,K83E,0.0622,2367,0-5 years
733,Y94N,-0.0583,2367,0-5 years
934,A128T,-0.0154,2367,0-5 years
953,T131K,0.0061,2367,0-5 years
...,...,...,...,...
1209,L164Q,0.0016,18C,40-45 years
1307,G186D,0.1335,18C,40-45 years
1373,D190N,-0.1872,18C,40-45 years
1431,Y195F,0.0371,18C,40-45 years


In [32]:
# filtered sites, scatterplot
scatterplot = (
    alt.Chart()
    .encode(
        x=alt.X("mutation", 
                title="mutation",
               ),
        y=alt.Y(
            "escape_mean",
            title="escape score",
        ),
        xOffset='jitter:Q',
        color=alt.Color('age_group:N', 
                        legend=None
                       ).scale(scheme='set2'),
        detail='serum',
        tooltip=['serum', 'escape_mean']
    ).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")
    .mark_circle(size=40, opacity=0.7)
    .properties(width=700, height=200)
)

x_axis = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(opacity=0.5).encode(y='y')

faceted_scatter = alt.layer(
    scatterplot, x_axis, data=full_escape
).configure_axis(
    grid=False,
    labelFontSize=13,
    titleFontSize=15
)

# faceted_scatter_filtered.save('scratch_notebooks/figure_drafts/sitewise_escape_plots/230714_summed_escape_scatterplot.html')

faceted_scatter

alt.LayerChart(...)

In [33]:
# filtered sites, scatterplot
scatterplot = (
    alt.Chart()
    .encode(
        x=alt.X("mutation", 
                title="mutation",
               ),
        y=alt.Y(
            "escape_mean",
            title="escape score",
        ),
        # xOffset='jitter:Q',
        color=alt.Color('age_group:N', 
                        legend=None
                       ).scale(scheme='set2'),
        detail='serum',
        tooltip=['serum', 'escape_mean']
    )
    .mark_circle(size=40, opacity=0.7)
    .properties(width=600, height=150)
)

x_axis = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(opacity=0.5).encode(y='y')

faceted_scatter = alt.layer(
    scatterplot, x_axis, data=full_escape
).facet(
    facet=alt.Facet(
        'age_group:N',
        title='escape at Darwin/2021 mutations',
        header=alt.Header(
            titleFontSize=21,
            titleFontWeight='normal',
            titlePadding=5,
            labelFontSize=17,
            labelOrient='right',
            # labelAngle=0,
            labelFontStyle='italic'
        )
    ),
    columns=1,
    spacing=5
).configure_axis(
    grid=False,
    labelFontSize=13,
    titleFontSize=15
)

# faceted_scatter_filtered.save('scratch_notebooks/figure_drafts/sitewise_escape_plots/230714_summed_escape_scatterplot.html')

faceted_scatter

alt.FacetChart(...)

In [34]:
# filtered sites, scatterplot
lineplot = (
    alt.Chart()
    .encode(
        x=alt.X("mutation", 
                title="mutation",
               ),
        y=alt.Y(
            "escape_mean",
            title="escape score",
        ),
        color=alt.Color('age_group:N', 
                        legend=None
                       ).scale(scheme='set2'),
        detail='serum',
        tooltip=['serum', 'escape_mean']
    )
    .mark_line(size=1, opacity=0.7)
    .properties(width=600, height=150)
)

scatterplot = (
    alt.Chart()
    .encode(
        x=alt.X("mutation", 
                title="mutation",
               ),
        y=alt.Y(
            "escape_mean",
            title="escape score",
        ),
        # xOffset='jitter:Q',
        color=alt.Color('age_group:N', 
                        legend=None
                       ).scale(scheme='set2'),
        detail='serum',
        tooltip=['serum', 'escape_mean']
    )
    .mark_circle(size=30, opacity=0.7)
    .properties(width=600, height=150)
)

x_axis = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(opacity=0.5).encode(y='y')

faceted_scatter = alt.layer(
    scatterplot, lineplot, x_axis, data=full_escape
).facet(
    facet=alt.Facet(
        'age_group:N',
        title='escape at Darwin/2021 mutations',
        header=alt.Header(
            titleFontSize=21,
            titleFontWeight='normal',
            titlePadding=5,
            labelFontSize=17,
            labelOrient='right',
            # labelAngle=0,
            labelFontStyle='italic'
        )
    ),
    columns=1,
    spacing=5
).configure_axis(
    grid=False,
    labelFontSize=13,
    titleFontSize=15
)

# faceted_scatter_filtered.save('scratch_notebooks/figure_drafts/sitewise_escape_plots/230714_summed_escape_scatterplot.html')

faceted_scatter

alt.FacetChart(...)